In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
import tensorflow as tf
from ViT import ViT

#### Hyperparameters

In [3]:
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 32
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 5
hp["class_names"] = ["daisy", "dandelion", "rose", "sunflower", "tulip"]

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 3072
hp["num_heads"] = 12
hp["dropout_rate"] = 0.1

In [4]:
def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)
		
		
def load_data(path, split=0.1):
	images = shuffle(glob(os.path.join(path, "*", "*.jpg")))
	# print(images)
	split_size = int(len(images) * split)
	# print(split_size)
	train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
	train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
	
	return train_x, valid_x, test_x


def process_image_label(path):
	"""Reading images"""
	path = path.decode()
	image = cv2.imread(path, cv2.IMREAD_COLOR)
	image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
	image = image/255.0
	# print(image.shape)
	
	""" Preprocessing to patches """
	patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
	patches = patchify(image, patch_shape, hp["patch_size"])
	# print(patches.shape)
	
	# patches = np.reshape(patches, (64, 25, 25, 3))
	# for i in range(64):
	# 	cv2.imwrite(f"./files/{i}.png", patches[i])
	
	patches = np.reshape(patches, hp["flat_patches_shape"])
	patches = patches.astype(np.float32)
	
	
	""" Label """
	class_name = path.split("/")[-2]
	class_index = hp["class_names"].index(class_name)
	class_index = np.array(class_index, dtype=np.int32)
	
	return patches, class_index


def parse(path):
	patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
	labels = tf.one_hot(labels, hp["num_classes"])
	
	patches.set_shape(hp["flat_patches_shape"])
	labels.set_shape(hp["num_classes"])
	
	return patches, labels


def tf_dataset(images, batch=32):
	ds = tf.data.Dataset.from_tensor_slices((images))
	ds = ds.map(parse).batch(batch).prefetch(8)
	return ds

### Seeding

In [5]:
np.random.seed(42)
tf.random.set_seed(42)

### "i" Directory for storing files

In [6]:
create_dir("files")

# Path to Dataset
dataset_path = os.path.join("flower_photos")
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "log.csv")

In [7]:
train_x, valid_x, test_x = load_data(dataset_path)
print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

Train: 2936 - Valid: 367 - Test: 367


In [9]:
train_ds = tf_dataset(train_x, batch=hp["batch_size"])
valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])

### Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

model = ViT(hp)
model.compile(
	loss="categorical_crossentropy",
	optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
	metrics=["accuracy"]
)

callbacks = [
	ModelCheckpoint(model_path, monitor="val_loss", verbose=1, save_best_only=True),
	ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10, min_lr=1e-1, verbose=1),
	CSVLogger(csv_path),
	EarlyStopping(monitor="val_loss", patience=50, restore_best_weights=False)
]

In [ ]:
model.fit(
	train_ds,
    epochs=hp["num_epochs"],
	validation_data=valid_ds,
	callbacks=callbacks
)